# Oral Disease Detection - Dataset Exploration

This notebook explores the oral diseases dataset from Kaggle.

**DISCLAIMER:** This AI provides preliminary screening only and is not a medical diagnosis.

In [ ]:
# Install required packages
!pip install kagglehub matplotlib seaborn pillow numpy pandas -q

In [ ]:
import kagglehub
import os
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
import numpy as np
import pandas as pd

# Download dataset
print("Downloading dataset...")
dataset_path = kagglehub.dataset_download("salmansajid05/oral-diseases")
print(f"Dataset path: {dataset_path}")

In [ ]:
# Explore directory structure
data_dir = Path(dataset_path)

print("Directory structure:")
for item in data_dir.rglob('*'):
    if item.is_dir():
        print(f"  📁 {item.relative_to(data_dir)}")

In [ ]:
# Find data root (directory containing class folders)
def find_data_root(base_path):
    for subdir in Path(base_path).iterdir():
        if subdir.is_dir():
            # Check if this directory contains image files
            images = list(subdir.glob('*.jpg')) + list(subdir.glob('*.png'))
            if images:
                return base_path
            # Check subdirectories
            for sub in subdir.iterdir():
                if sub.is_dir():
                    images = list(sub.glob('*.jpg')) + list(sub.glob('*.png'))
                    if images:
                        return subdir
    return base_path

data_root = find_data_root(dataset_path)
print(f"Data root: {data_root}")

In [ ]:
# Analyze classes
class_info = {}

for class_dir in sorted(Path(data_root).iterdir()):
    if class_dir.is_dir():
        images = list(class_dir.glob('*.jpg')) + list(class_dir.glob('*.jpeg')) + list(class_dir.glob('*.png'))
        images += list(class_dir.glob('*.JPG')) + list(class_dir.glob('*.JPEG')) + list(class_dir.glob('*.PNG'))
        class_info[class_dir.name] = {
            'count': len(images),
            'path': class_dir,
            'images': images
        }

# Create DataFrame
df = pd.DataFrame([
    {'Class': name, 'Count': info['count']}
    for name, info in class_info.items()
])

print("\nClass Distribution:")
print(df.to_string(index=False))
print(f"\nTotal images: {df['Count'].sum()}")

In [ ]:
# Visualize class distribution
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
sns.barplot(data=df, x='Class', y='Count', palette='viridis')
plt.xticks(rotation=45, ha='right')
plt.title('Images per Class')
plt.xlabel('Disease Class')
plt.ylabel('Number of Images')

plt.subplot(1, 2, 2)
plt.pie(df['Count'], labels=df['Class'], autopct='%1.1f%%', colors=sns.color_palette('viridis', len(df)))
plt.title('Class Distribution')

plt.tight_layout()
plt.savefig('../results/class_distribution.png', dpi=150)
plt.show()

In [ ]:
# Check for class imbalance
counts = df['Count'].values
imbalance_ratio = max(counts) / min(counts) if min(counts) > 0 else float('inf')

print(f"Class imbalance ratio: {imbalance_ratio:.2f}")
if imbalance_ratio > 3:
    print("⚠️ Significant class imbalance detected!")
    print("Recommendations:")
    print("  - Use class weights during training")
    print("  - Apply data augmentation to minority classes")
    print("  - Consider oversampling/undersampling")
else:
    print("✓ Class distribution is relatively balanced")

In [ ]:
# Display sample images from each class
n_classes = len(class_info)
samples_per_class = 4

fig, axes = plt.subplots(n_classes, samples_per_class, figsize=(16, 4*n_classes))

for idx, (class_name, info) in enumerate(sorted(class_info.items())):
    images = info['images'][:samples_per_class]
    
    for j in range(samples_per_class):
        ax = axes[idx, j] if n_classes > 1 else axes[j]
        
        if j < len(images):
            img = Image.open(images[j])
            ax.imshow(img)
            if j == 0:
                ax.set_ylabel(class_name, fontsize=12, rotation=0, ha='right', va='center')
        ax.axis('off')

plt.suptitle('Sample Images from Each Disease Class', fontsize=16, y=1.02)
plt.tight_layout()
plt.savefig('../results/sample_images.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# Analyze image properties
image_stats = []

for class_name, info in class_info.items():
    for img_path in info['images'][:50]:  # Sample 50 images per class
        try:
            img = Image.open(img_path)
            image_stats.append({
                'class': class_name,
                'width': img.size[0],
                'height': img.size[1],
                'mode': img.mode,
                'aspect_ratio': img.size[0] / img.size[1]
            })
        except Exception as e:
            print(f"Error reading {img_path}: {e}")

stats_df = pd.DataFrame(image_stats)

print("\nImage Statistics:")
print(f"Width range: {stats_df['width'].min()} - {stats_df['width'].max()}")
print(f"Height range: {stats_df['height'].min()} - {stats_df['height'].max()}")
print(f"Average size: {stats_df['width'].mean():.0f} x {stats_df['height'].mean():.0f}")
print(f"\nImage modes: {stats_df['mode'].value_counts().to_dict()}")

In [ ]:
# Visualize image size distribution
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

axes[0].hist(stats_df['width'], bins=30, color='steelblue', edgecolor='white')
axes[0].set_title('Width Distribution')
axes[0].set_xlabel('Width (pixels)')

axes[1].hist(stats_df['height'], bins=30, color='coral', edgecolor='white')
axes[1].set_title('Height Distribution')
axes[1].set_xlabel('Height (pixels)')

axes[2].hist(stats_df['aspect_ratio'], bins=30, color='seagreen', edgecolor='white')
axes[2].set_title('Aspect Ratio Distribution')
axes[2].set_xlabel('Aspect Ratio')

plt.tight_layout()
plt.savefig('../results/image_stats.png', dpi=150)
plt.show()

## Summary

Based on the exploration:

1. **Dataset contains multiple oral disease classes**
2. **Class imbalance may need to be addressed**
3. **Images have varying sizes - will resize to 224x224**
4. **Data augmentation will help improve model generalization**

Next step: Run `train.py` to train the model.